> __Purpose:__ Implement an initial privacy attack to quantify how much re-identiifcation and linkability risk exists from personalization parameters (the decoder used in the co-adaptation algorithm). Thus, implement a basic ML model to link the decoder matrices back to the corresponding subject.  

- 14 Subjects, therefore random guessing would be 7.14% correct on average

In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import seaborn
import time
import pandas as pd
import pickle
import random
from kcs_ml_infr import *
from experiment_params import *

random.seed(a=1)

# Load In Data

I'm just making the flatten array input DF here

In [2]:
update_ix = np.load(r"Data\update_ix.npy")
print(update_ix.shape)
print(update_ix)

dec_flattened_df1 = pd.read_csv("Data\dec_full_flattened_df1.csv")
dec_flattened_df2 = pd.read_csv("Data\dec_full_flattened_df2.csv")

dec_norms_df1 = pd.read_csv("Data\decoder_full_norms1.csv")
dec_norms_df2 = pd.read_csv("Data\decoder_full_norms2.csv")

(19,)
[    0  1200  2402  3604  4806  6008  7210  8412  9614 10816 12018 13220
 14422 15624 16826 18028 19230 20432 20769]


In [3]:
dec_norms_df = pd.concat((dec_norms_df1, dec_norms_df2))
dec_flattened_df = pd.concat((dec_flattened_df1, dec_flattened_df2))
if dec_flattened_df.columns[0]=='Unnamed: 0':
    dec_flattened_df.drop('Unnamed: 0', axis=1, inplace=True)

# This one isn't working yet
#dec_vec_norm_df = pd.concat((dec_vec_norm_df1, dec_vec_norm_df2))

print(dec_flattened_df.shape)
dec_flattened_df.head()

(2128, 4)


,Subject,Condition,Update Number,Flattened Decoder
0,METACPHS_S106,0,0,[2.72214435e-03 2.60593156e-03 3.08748960e-03 ...
1,METACPHS_S106,0,1,[2.72214435e-03 2.60593156e-03 3.08748960e-03 ...
2,METACPHS_S106,0,2,[-0.34157085 0.08410593 -0.54057447 0.444319...
3,METACPHS_S106,0,3,[-0.17386248 0.00370517 0.40721562 -0.515221...
4,METACPHS_S106,0,4,[ 0.83210615 -1.32612423 0.28718983 1.302252...


In [4]:
print(dec_norms_df.shape)
dec_norms_df.drop('Unnamed: 0', axis=1, inplace=True)
dec_norms_df.head()

(2128, 5)


,Subject,Condition,Update Number,Frobenius Norm
0,METACPHS_S106,0,0,0.066360
1,METACPHS_S106,0,1,0.066360
2,METACPHS_S106,0,2,9.709390
3,METACPHS_S106,0,3,8.209080
4,METACPHS_S106,0,4,10.406943


# Classification
1. Logistic Regression
2. K-Nearest Neighbor
3. Gaussian Naive Bayes
4. Linear SVC
5. Stochastic Gradient Descent
6. Decision Tree Classifier
7. Gradient Boosting Trees

In [5]:
# Machine learning
from sklearn.model_selection import train_test_split
from sklearn import model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier

## 1) Linking Matrix Norms to Participants

In [7]:
norm_label_df = dec_norms_df.drop(["Condition", "Update Number", "Frobenius Norm"], axis=1)

In [8]:
key_to_num = dict()
num_to_key = dict()
for idx, key in enumerate(keys):
    key_to_num[key] = idx
    num_to_key[idx] = key

In [9]:
norm_label_df["Subject"] = norm_label_df["Subject"].map(key_to_num)
norm_label_df.head()

,Subject
0,0
1,0
2,0
3,0
4,0


In [10]:
# Result logs
my_metrics_cols = ['Algorithm', 'One Off Acc', 'CV Acc', 'K Folds']
res_df = pd.DataFrame(columns=my_metrics_cols)

norm1d_res_df = pd.DataFrame(columns=my_metrics_cols)
SSnorm1d_res_df = pd.DataFrame(columns=my_metrics_cols)
flatten_dec_res_df = pd.DataFrame(columns=my_metrics_cols)
SSflatten_dec_res_df = pd.DataFrame(columns=my_metrics_cols)
norm_vec_res_df = pd.DataFrame(columns=my_metrics_cols)
SSnorm_vec_res_df = pd.DataFrame(columns=my_metrics_cols)

In [11]:
my_models = [LogisticRegression(), KNeighborsClassifier(), GaussianNB(), LinearSVC(), SGDClassifier(), DecisionTreeClassifier(), GradientBoostingClassifier()]

## Case 1: 1D Norm

In [12]:
norm_input_df = dec_norms_df.drop(["Subject",  "Condition", "Update Number"], axis=1)
norm_input_df.head()

,Frobenius Norm
0,0.066360
1,0.066360
2,9.709390
3,8.209080
4,10.406943


In [13]:
X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(norm_input_df, norm_label_df)
y_train = np.ravel(y_train)

print(X_train.shape)
X_train.head()

(1489, 1)


,Frobenius Norm
420,3.567582
146,5.589772
266,0.069974
695,5.796815
372,17.938752


In [14]:
for model_num, model in enumerate(my_models):
    norm1d_res_df = train_model(model, X_train, y_train, cv, norm1d_res_df)

C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [15]:
norm1d_res_df.head(100)

,Algorithm,One Off Acc,CV Acc,K Folds,N
0,LogisticRegression(),17.931,16.857,10,NA
0,KNeighborsClassifier(),52.048,30.356,10,NA
0,GaussianNB(),19.073,17.260,10,NA
0,LinearSVC(),18.737,17.461,10,NA
0,SGDClassifier(),18.670,15.782,10,NA
0,DecisionTreeClassifier(),100.000,28.946,10,NA
0,GradientBoostingClassifier(),65.682,32.304,10,NA


Now Test

In [16]:
test_df = pd.DataFrame(columns=['Algorithm', 'CV Acc', 'Test Acc', 'K Folds'])

for model_num, model in enumerate(my_models):
    test_df = test_model(model, X_train, y_train, X_test, y_test, test_df, cv)

C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepd

In [17]:
test_df.head(100)

,Algorithm,CV Acc,Test Acc,K Folds,N
0,LogisticRegression(),16.857,16.901,10,NA
0,KNeighborsClassifier(),30.356,28.795,10,NA
0,GaussianNB(),17.26,19.718,10,NA
0,LinearSVC(),18.603,18.466,10,NA
0,SGDClassifier(),14.305,19.405,10,NA
0,DecisionTreeClassifier(),28.946,27.856,10,NA
0,GradientBoostingClassifier(),32.304,27.23,10,NA


## Case 2: Flattened Decoder Matrices as Input

In [18]:
flattened_input_df = dec_flattened_df.drop(["Subject",  "Condition", "Update Number"], axis=1)
flattened_input_df.head()

,Flattened Decoder
0,[2.72214435e-03 2.60593156e-03 3.08748960e-03 ...
1,[2.72214435e-03 2.60593156e-03 3.08748960e-03 ...
2,[-0.34157085 0.08410593 -0.54057447 0.444319...
3,[-0.17386248 0.00370517 0.40721562 -0.515221...
4,[ 0.83210615 -1.32612423 0.28718983 1.302252...


Since it was loaded in from a csv, the list in the column needs to be coverted from a string back to a list

In [19]:
#df_obj.apply(lambda x: x.str.strip())
flattened_input_df["List Flat Dec"] = flattened_input_df["Flattened Decoder"].apply(lambda x: x.strip("[]").split())
flattened_input_df.head()

,Flattened Decoder,List Flat Dec
0,[2.72214435e-03 2.60593156e-03 3.08748960e-03 ...,"[2.72214435e-03, 2.60593156e-03, 3.08748960e-0..."
1,[2.72214435e-03 2.60593156e-03 3.08748960e-03 ...,"[2.72214435e-03, 2.60593156e-03, 3.08748960e-0..."
2,[-0.34157085 0.08410593 -0.54057447 0.444319...,"[-0.34157085, 0.08410593, -0.54057447, 0.44431..."
3,[-0.17386248 0.00370517 0.40721562 -0.515221...,"[-0.17386248, 0.00370517, 0.40721562, -0.51522..."
4,[ 0.83210615 -1.32612423 0.28718983 1.302252...,"[0.83210615, -1.32612423, 0.28718983, 1.302252..."


In [20]:
flattened_input_df.drop("Flattened Decoder", axis=1, inplace=True)
flattened_input_df.head()

,List Flat Dec
0,"[2.72214435e-03, 2.60593156e-03, 3.08748960e-0..."
1,"[2.72214435e-03, 2.60593156e-03, 3.08748960e-0..."
2,"[-0.34157085, 0.08410593, -0.54057447, 0.44431..."
3,"[-0.17386248, 0.00370517, 0.40721562, -0.51522..."
4,"[0.83210615, -1.32612423, 0.28718983, 1.302252..."


In [21]:
X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(flattened_input_df, norm_label_df)
y_train = np.ravel(y_train)

print(X_train.shape)
X_train.head()

(1489, 1)


,List Flat Dec
420,"[0.559448, 0.48120126, -0.31172232, -0.3090073..."
146,"[-0.05469056, 0.08547178, -0.13961235, 0.10982..."
266,"[-8.97360423e-03, -8.31381090e-03, -6.41220148..."
695,"[-0.05559843, -1.09936482, 0.04076854, -0.2880..."
372,"[2.09767182, -1.4354094, -0.05695288, 1.670269..."


In [22]:
X_train2 = pd.DataFrame()
X_test2 = pd.DataFrame()
for my_row in range(X_train.shape[0]):
    test=pd.DataFrame(X_train.iloc[my_row,0]).T
    X_train2 = pd.concat((X_train2, test))
    
for my_row in range(X_test.shape[0]):
    test=pd.DataFrame(X_test.iloc[my_row,0]).T
    X_test2 = pd.concat((X_test2, test))
    
X_train = X_train2
X_test = X_test2
    
X_train2.head()

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,0.559448,0.48120126,-0.31172232,-0.30900738,0.10670872,0.16178469,-0.30169939,-0.32196656,-0.08306045,0.57048804,...,0.03931518,0.36154944,0.4114088,0.18165392,0.16846536,0.04893817,0.07089201,-0.0321285,0.05558241,0.4665362
0,-0.05469056,0.08547178,-0.13961235,0.10982815,-0.13220538,-0.12586283,0.87265727,0.05764217,1.21553498,-0.04298447,...,0.29588525,-0.66453095,-0.83194619,1.21445527,0.90520372,-1.33284078,-0.03771546,0.54090509,0.18123141,-0.61762233
0,-8.97360423e-03,-8.31381090e-03,-6.41220148e-03,-1.12539235e-03,-3.36881772e-03,-1.68942685e-03,-8.62278986e-03,-4.55590985e-03,-9.38837743e-03,-2.21027321e-03,...,-8.42358503e-03,-2.35925820e-03,-1.52956553e-03,-2.41583688e-03,-1.75725760e-03,-7.07813716e-03,-7.92085039e-03,-6.52980615e-03,-3.72468265e-03,-5.48093337e-03
0,-0.05559843,-1.09936482,0.04076854,-0.2880755,0.01606553,-0.11349792,0.08254973,0.0190325,-0.09000466,-1.67006695,...,0.00336473,0.3465541,-0.18919889,0.18989056,0.21080283,0.02220322,-0.02149587,0.43491206,-0.83281655,0.03452957
0,2.09767182,-1.4354094,-0.05695288,1.67026996,-1.50149684,-0.96235093,0.08352334,2.7330404,-1.27054454,0.09729616,...,-2.02883978,-0.35463954,2.23102033,2.79286394,-1.5247046,2.49386391,-0.64878113,-1.36914659,1.04705914,-0.72075387


Convert single element of a vector into a vector of single elements!

In [23]:
for model_num, model in enumerate(my_models):
    print(f"{model_num} of {len(my_models)}")
    flatten_dec_res_df = train_model(model, X_train, y_train, cv, flatten_dec_res_df)

0 of 7
1 of 7


C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


2 of 7
3 of 7


C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


4 of 7
5 of 7
6 of 7


In [24]:
flatten_dec_res_df.head(100)

,Algorithm,One Off Acc,CV Acc,K Folds,N
0,LogisticRegression(),87.911,70.853,10,NA
0,KNeighborsClassifier(),83.815,76.494,10,NA
0,GaussianNB(),71.592,69.778,10,NA
0,LinearSVC(),83.277,70.987,10,NA
0,SGDClassifier(),77.905,68.704,10,NA
0,DecisionTreeClassifier(),100.000,66.488,10,NA
0,GradientBoostingClassifier(),100.000,87.979,10,NA


Test the models on the testing data

In [25]:
test_df = pd.DataFrame(columns=['Algorithm', 'CV Acc', 'Test Acc', 'K Folds'])
for model_num, model in enumerate(my_models):
    print(f"{model_num+1} of {len(my_models)}")
    test_df = test_model(model, X_train, y_train, X_test, y_test, test_df, cv)

1 of 7
2 of 7


C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepd

3 of 7
4 of 7


C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


5 of 7
6 of 7
7 of 7


In [26]:
test_df.head(100)

,Algorithm,CV Acc,Test Acc,K Folds,N
0,LogisticRegression(),70.853,71.987,10,NA
0,KNeighborsClassifier(),76.494,76.526,10,NA
0,GaussianNB(),69.778,72.926,10,NA
0,LinearSVC(),71.054,74.022,10,NA
0,SGDClassifier(),68.502,71.049,10,NA
0,DecisionTreeClassifier(),66.488,67.136,10,NA
0,GradientBoostingClassifier(),87.844,90.141,10,NA


Redo but try using Standard Scaler this time

In [27]:
from sklearn.preprocessing import StandardScaler

scaler_Xtrain = StandardScaler().fit(X_train)
XtrainSS = scaler_Xtrain.transform(X_train)

print(X_train.shape)
print(XtrainSS.shape)

(1489, 128)
(1489, 128)


In [28]:
for model_num, model in enumerate(my_models):
    print(f"{model_num+1} of {len(my_models)}")
    SSflatten_dec_res_df = train_model(model, XtrainSS, y_train, cv, SSflatten_dec_res_df)

1 of 7
2 of 7


C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\neighbors\_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


3 of 7
4 of 7


C:\Users\kdmen\miniconda3\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


5 of 7
6 of 7
7 of 7


In [29]:
SSflatten_dec_res_df.head(100)

,Algorithm,One Off Acc,CV Acc,K Folds,N
0,LogisticRegression(),89.322,70.114,10,NA
0,KNeighborsClassifier(),82.740,74.211,10,NA
0,GaussianNB(),71.592,69.778,10,NA
0,LinearSVC(),83.277,71.189,10,NA
0,SGDClassifier(),79.181,67.562,10,NA
0,DecisionTreeClassifier(),100.000,65.816,10,NA
0,GradientBoostingClassifier(),100.000,88.449,10,NA


Thus we see no real performance gains by using standard scaler.

Now try and optimize the best performer (GBT)
1. max_depth: int, default=3
- The maximum depth of the individual regression estimators. The maximum depth limits the number of nodes in the tree. Tune this parameter for best performance; the best value depends on the interaction of the input variables. Values must be in the range [1, inf).

## Case 3: Feed in a Vector of Norms
> This isn't super high priority since the flattened version has high accuracy
> Ideally, we would be able to use norms or DFT coefficients to achieve the same purpose.  If it still can link then that would be bad, for our specific application (eg the decoder matrix), DFT coeffs and heavily windowed data and/or norms cannot be used in the convex optimziation problem and thus this doesn't have a clear point for now)

In [30]:
'''
#dec_vec_norm_df
dec_vec_norm_input_df = dec_vec_norm_df.drop(["Subject",  "Condition"], axis=1)
dec_vec_norm_input_df.head()

# norm_label_df probably needs to be changed to a new variable... shouldn't have the same shape...
X_train, y_train, X_test, y_test, X_val, y_val = train_test_val_split(norm_input_df, norm_label_df)
y_train = np.ravel(y_train)

print(X_train.shape)
X_train.head()

X_train2 = pd.DataFrame()
X_test2 = pd.DataFrame()
for my_row in range(X_train.shape[0]):
    test=pd.DataFrame(X_train.iloc[my_row,0]).T
    X_train2 = pd.concat((X_train2, test))
    
for my_row in range(X_test.shape[0]):
    test=pd.DataFrame(X_test.iloc[my_row,0]).T
    X_test2 = pd.concat((X_test2, test))
X_train2.head()

for model_num, model in enumerate(my_models):
    print(f"{model_num} of {len(my_models)}")
    norm_vec_res_df = train_model(model, X_train2, y_train, cv, norm_vec_res_df)
    
norm_vec_res_df.head(100)
'''
0

0